In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q

In [2]:
!pip install openai-whisper ffmpeg-python -q
!pip install ffmpeg -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!apt-get update && apt-get install -y ffmpeg
!ffmpeg -version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,563 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Get:13 http://archive.ubuntu.com/u

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

path = "drive/MyDrive/Speech2Text/dataset/"

if not os.path.exists(path):
    try:
        os.makedirs(path)
        print(f"The directory '{path}' has been created successfully.")
    except OSError as e:
        print(f"Error creating directory: {e}")
else:
    print(f"The directory '{path}' already exists.")

The directory 'drive/MyDrive/Speech2Text/dataset/' already exists.


In [6]:
import os

file_path = "drive/MyDrive/Speech2Text/dataset/grandfather._2015-10-09_202035.mp3"
print("File exists:", os.path.exists(file_path))

File exists: True


In [7]:
import subprocess

subprocess.call(['ffmpeg', '-i', f"{path}/grandfather._2015-10-09_202035.mp3",
                 f'{path}/converted_to_wav_file.wav'])

1

In [8]:
!ffmpeg -i "drive/MyDrive/Speech2Text/dataset/grandfather._2015-10-09_202035.mp3" -ar 16000 -ac 1 "drive/MyDrive/Speech2Text/dataset/converted_to_wav_file.wav"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [9]:
import subprocess

def convert_to_wav(input_file, output_file):
    """
    Converts an audio file to WAV format with 16kHz sampling rate and mono channel using ffmpeg.

    Args:
        input_file (str): Path to the input audio file.
        output_file (str): Path to save the converted WAV file.
    """
    try:
        # Run the ffmpeg command
        command = [
            "ffmpeg",
            "-i", input_file,         # Input file
            "-ar", "16000",           # Sampling rate: 16kHz
            "-ac", "1",               # Channels: mono
            output_file               # Output file
        ]
        subprocess.run(command, check=True)
        print(f"Conversion successful! WAV file saved at: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")
    except FileNotFoundError:
        print("Error: ffmpeg is not installed or not in your PATH.")

# Example Usage
if __name__ == "__main__":
    # Define the input and output file paths
    input_path = file_path
    output_path = "drive/MyDrive/Speech2Text/dataset/converted_to_wav_file.wav"

    # Convert the file to WAV format
    convert_to_wav(input_path, output_path)

Error during conversion: Command '['ffmpeg', '-i', 'drive/MyDrive/Speech2Text/dataset/grandfather._2015-10-09_202035.mp3', '-ar', '16000', '-ac', '1', 'drive/MyDrive/Speech2Text/dataset/converted_to_wav_file.wav']' returned non-zero exit status 1.


In [10]:
import whisper
import warnings
import csv

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

def transcribe_persian_audio(audio_file, output_csv="transcription_output.csv", use_gpu=False):
    """
    Transcribes Persian speech from an audio file using the Whisper model and saves the output to a CSV file.

    Args:
        audio_file (str): Path to the audio file to transcribe.
        output_csv (str): Path to save the transcription and timestamps as a CSV file.
        use_gpu (bool): If True, use GPU for faster transcription.

    Returns:
        None
    """
    try:
        # Set the device: "cuda" for GPU, "cpu" for CPU
        device = "cuda" if use_gpu else "cpu"

        # Load the Whisper model
        print(f"Loading Whisper model on {'GPU' if use_gpu else 'CPU'}...")
        model = whisper.load_model("large", device=device)

        # Transcribe the audio with Persian language specified
        print("Transcribing audio...")
        result = model.transcribe(audio_file, language="fa", task="transcribe")

        # Extract transcription segments with timestamps
        segments = result.get('segments', [])
        transcription_data = []

        for segment in segments:
            start = segment['start']  # Start time of the segment
            end = segment['end']      # End time of the segment
            text = segment['text']    # Transcribed text
            transcription_data.append([start, end, text])

        # Save to a CSV file
        with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Start Time (s)", "End Time (s)", "Text"])  # Write headers
            writer.writerows(transcription_data)

        print(f"Transcription saved to {output_csv}")

    except Exception as e:
        print(f"An error occurred during transcription: {e}")

if __name__ == "__main__":
    # Specify the path to your Persian audio file
    audio_path = f'{path}/converted_to_wav_file.wav'  # Replace with the path to your audio file
    output_csv_path = f"{path}/transcription_output.csv"  # Specify the output CSV file name

    # Choose whether to use GPU
    use_gpu = False  # Change to True if you have a compatible GPU

    try:
        # Call the transcription function
        transcribe_persian_audio(audio_path, output_csv=output_csv_path, use_gpu=use_gpu)
    except FileNotFoundError:
        print("Error: The specified audio file was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Loading Whisper model on CPU...


100%|█████████████████████████████████████| 2.88G/2.88G [01:08<00:00, 45.1MiB/s]


Transcribing audio...
Transcription saved to drive/MyDrive/Speech2Text/dataset//transcription_output.csv


In [11]:
import whisper
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

def transcribe_persian_audio(audio_file, use_gpu=False):
    """
    Transcribes Persian speech from an audio file using the Whisper model.

    Args:
        audio_file (str): Path to the audio file to transcribe.
        use_gpu (bool): If True, use GPU for faster transcription.

    Returns:
        str: The transcribed Persian text.
    """
    try:
        # Set the device: "cuda" for GPU, "cpu" for CPU
        device = "cuda" if use_gpu else "cpu"

        # Load the Whisper model
        print(f"Loading Whisper model on {'GPU' if use_gpu else 'CPU'}...")
        model = whisper.load_model("large", device=device)  # Replace "base" with "large" for better accuracy

        # Transcribe the audio with Persian language specified
        print("Transcribing audio...")
        result = model.transcribe(audio_file, language="fa")  # 'fa' is the language code for Persian

        # Extract the transcribed text
        transcript = result.get('text', '')
        return transcript

    except Exception as e:
        return f"An error occurred during transcription: {e}"

if __name__ == "__main__":
    # Specify the path to your Persian audio file
    audio_path = f'{path}/converted_to_wav_file.wav' # Replace with the path to your audio file

    # Choose whether to use GPU
    use_gpu = False  # Change to True if you have a compatible GPU

    try:
        # Call the transcription function
        transcription = transcribe_persian_audio(audio_path, use_gpu=use_gpu)

        # Display the transcription result
        print("\nTranscription Output:")
        print(transcription)
    except FileNotFoundError:
        print("Error: The specified audio file was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Loading Whisper model on CPU...
Transcribing audio...

Transcription Output:
 باستاشه بودند تا خونه شون ساخته اینجا پچه خونه بینید بله اینجا بودند خدا نقد کنید مو مو مادر ایادو توخته بینید خدا سرخواهید من گوشت که مادر بود دختر است همه در اینجا داشته مادر گفته که از اون طریقه هم مادر باشه نه بشه مادر سرخواهید که از جندازش بیدارند من گوشت مادر خدمت برنامه اصلا از اون من خان صبح منو تازه دارم ترک بردم منو خودت بزرگ شده تکتیف نمی کنم صبح میستم پامشه صحبه باشه خانه می کرده خانه بر بیاده فرار آتش می کرده فرار کشم می کرده اونو می پخدید موقعی صبحانه خورده بهشون اونو تازه می برده صبح این کشم imposی و گوش حتی دیگر ساخی جیا داره په PCB مادر سفر که از پای ماهيد درها باشه این بیای می کنه چیزا برای بتوAUان خوبیه این بیای mereka خود نمایم دلش نمی بند سویده نظرشو برداشت بود همه اشتخاصی اینجا بکنه یعنی اینقدر نورانی بود؟ بله خدا محمود شما خیلی بله چقدر داغ دارید شما بله ماما، مادر بزرگ این گفتن چهار تا خب هم دوتا پدر بزرگ هم دوتا دختراشون جوانین بودن که فوت شدن گفتن سر فوته دیگه آقام ازن چشمشون دیگه کو